# St. Jude Data Consolidation and Analysis

This script processes and analyzes storage location mapping data from multiple sources.
It consolidates data from different sheets, filters relevant information, and distributes
the data across Cohesity storage systems.

In [1]:
import numpy as np
import pandas as pd

## Section 1: Data Loading and Initial Consolidation

This section:
- Loads the Excel file containing storage location mapping data
- Reads data from multiple sheets (RDDR01 through RDDR06)
- Combines all sheets into a single consolidated dataframe

In [2]:
# Load the Excel file
file_path = 'DR_Storage_Location_Mapping_V1.0.xlsx'
excel_data = pd.ExcelFile(file_path)

# List of sheet names to consolidate
sheet_names = ['RDDR01', 'RDDR02', 'RDDR03', 'RDDR04', 'RDDR05', 'RDDR06']

# Read and concatenate data from the specified sheets
data_frames = [excel_data.parse(sheet) for sheet in sheet_names]
consolidated_data = pd.concat(data_frames, ignore_index=True)

# Display the last few rows to verify the data
print("\nInitial consolidated data preview:")
print(consolidated_data.tail())


Initial consolidated data preview:
    Source System   Fileset Name Source Path Target Path Source GB 3/29  \
378          Jude  groups-zhugrp         NaN         NaN        34999.0   
379           NaN            NaN         NaN         NaN            NaN   
380           NaN  Groups Totals         NaN         NaN     12981433.0   
381           NaN            NaN         NaN         NaN            NaN   
382           NaN  RDDR06 Totals         NaN         NaN     12981433.0   

    Source Filecount 3/29 Target GB 3/29 Target Filecount 3/29 GB Complete  \
378              192066.0        34988.0                   NaN         NaN   
379                   NaN            NaN                   NaN         NaN   
380           651836097.0     12362228.0                   NaN         NaN   
381                   NaN            NaN                   NaN         NaN   
382           651836097.0     12362228.0                   NaN         NaN   

    File Complete  ...     Group Name Create

## Section 2: Data Cleaning and Filtering

This section:
- Removes rows with missing source paths
- Filters data to only include entries from 'Jude', 'RS1', or 'RS2' source systems
- Saves the cleaned data to a CSV file

In [3]:
# Remove rows with NaN in the 'source system' column
consolidated_data = consolidated_data.dropna(subset=['Source Path'])

# Drop rows that do not have 'Jude', 'RS1', or 'RS2' in the 'Source System' column
consolidated_data = consolidated_data[consolidated_data['Source System'].isin(['Jude', 'RS1', 'RS2'])]

# Save the consolidated data to a CSV file
output_csv_path = 'consolidated_data.csv'
consolidated_data.to_csv(output_csv_path, index=False)

print(f"\nConsolidated CSV file saved to {output_csv_path}")
print("\nCleaned data preview:")
print(consolidated_data.tail())


Consolidated CSV file saved to consolidated_data.csv

Cleaned data preview:
    Source System   Fileset Name                    Source Path Target Path  \
295          Jude   shres-IMINFO   /research/rgs01/shres/IMINFO         NaN   
296          Jude  shres-SCIMGEM  /research/rgs01/shres/SCIMGEM         NaN   
297          Jude  shres-SCIMGLM  /research/rgs01/shres/SCIMGLM         NaN   
298          Jude      shres-TGC      /research/rgs01/shres/TGC         NaN   
299          Jude     shres-VECT     /research/rgs01/shres/VECT         NaN   

    Source GB 3/29 Source Filecount 3/29 Target GB 3/29 Target Filecount 3/29  \
295       410143.0            36817327.0       405886.0            36428709.0   
296        37244.0             9767234.0        32778.0             9161474.0   
297       438176.0            10696449.0       424886.0            10269290.0   
298           21.0               11274.0           20.0               11194.0   
299         3112.0              708939.0   

## Section 3: Data Column Selection

This section:
- Selects only the essential columns: Source System, Source Path, and Source GB (3/29)
- Saves the filtered dataset to a new CSV file

In [4]:
# Keep only the specified columns
columns_to_keep = ['Source System', 'Source Path', 'Source GB 3/29']
data = consolidated_data[columns_to_keep]

# Save the data to a CSV file
output_csv_path = 'data.csv'
data.to_csv(output_csv_path, index=False)

print(f"\nFiltered data CSV file saved to {output_csv_path}")
print("\nSelected columns data preview:")
print(data.tail())


Filtered data CSV file saved to data.csv

Selected columns data preview:
    Source System                    Source Path Source GB 3/29
295          Jude   /research/rgs01/shres/IMINFO       410143.0
296          Jude  /research/rgs01/shres/SCIMGEM        37244.0
297          Jude  /research/rgs01/shres/SCIMGLM       438176.0
298          Jude      /research/rgs01/shres/TGC           21.0
299          Jude     /research/rgs01/shres/VECT         3112.0


## Section 4: Total Storage Calculation

This section:
- Sums up all values in the 'Source GB 3/29' column
- Converts the total from GB to PB (Petabytes)

In [5]:
# Sum the 'Source GB 3/29' column and convert from GB to PB
total_gb = data['Source GB 3/29'].sum()
total_pb = total_gb / 1_000_000

print(f"\nTotal Storage Calculation:")
print(f"Total GB: {total_gb:,.2f}")
print(f"Total PB: {total_pb:,.2f}")


Total Storage Calculation:
Total GB: 64,469,343.00
Total PB: 64.47


## Section 5: Cohesity Storage Distribution

This section:
- Creates 13 Cohesity storage groups
- Sorts data by storage size in descending order
- Distributes the data across Cohesity groups, aiming for approximately 5 PB per group
- Saves the distribution layout to a CSV file

In [6]:
# Divide data into 13 names for Cohesity column
names = ['Cohesity_' + str(i + 1) for i in range(13)]
data = data.sort_values(by='Source GB 3/29', ascending=False).reset_index(drop=True)
data['Cohesity'] = ''

# Allocate approximately 5 PB per Cohesity group
cohesity_sums = [0] * len(names)
for idx, row in data.iterrows():
    min_index = cohesity_sums.index(min(cohesity_sums))
    data.loc[idx, 'Cohesity'] = str(names[min_index])
    cohesity_sums[min_index] += row['Source GB 3/29']

# Save the grouped data to a CSV file
output_csv_path = 'layout.csv'
data.to_csv(output_csv_path, index=False)

print(f"\nGrouped data CSV file saved to {output_csv_path}")
print("\nDistribution preview:")
print(data.head())


Grouped data CSV file saved to layout.csv

Distribution preview:
  Source System                       Source Path Source GB 3/29    Cohesity
0           RS2       /research/rgs02/resgen/prod        4793325  Cohesity_1
1          Jude   /research/rgs01/groups/zhanggrp      4722804.0  Cohesity_2
2           RS1  /research/rgs01/home/clusterHome      4625031.5  Cohesity_3
3           RS1         /research/rgs01/reference      3174454.5  Cohesity_4
4          Jude   /research/rgs01/groups/mulligrp      3157182.0  Cohesity_5


## Section 6: Storage Distribution Summary

This section:
- Groups the data by Cohesity storage system
- Calculates the total storage allocated to each Cohesity system
- Converts the storage values from GB to PB
- Saves the summary to a CSV file

In [7]:
# Group by 'Cohesity' and sum 'Source GB 10/13', then convert to PB
grouped_data = data.groupby('Cohesity')['Source GB 3/29'].sum().reset_index()
grouped_data['Source PB'] = grouped_data['Source GB 3/29'] / 1_000_000

# Save the grouped data to a CSV file
output_csv_path = 'grouped_data.csv'
grouped_data.to_csv(output_csv_path, index=False)

print(f"\nGrouped data CSV file saved to {output_csv_path}")
print("\nStorage distribution summary:")
print(grouped_data)


Grouped data CSV file saved to grouped_data.csv

Storage distribution summary:
       Cohesity Source GB 3/29 Source PB
0    Cohesity_1      4959170.0   4.95917
1   Cohesity_10      4959176.0  4.959176
2   Cohesity_11      4959176.0  4.959176
3   Cohesity_12      4959181.0  4.959181
4   Cohesity_13      4959175.0  4.959175
5    Cohesity_2      4959170.0   4.95917
6    Cohesity_3      4959169.5  4.959169
7    Cohesity_4      4959219.5  4.959219
8    Cohesity_5      4959193.0  4.959193
9    Cohesity_6      4959171.0  4.959171
10   Cohesity_7      4959174.0  4.959174
11   Cohesity_8      4959169.0  4.959169
12   Cohesity_9      4959199.0  4.959199
